# Query for existing collections in 'embargo_or4'

 work with Weakly_2024_25
- use jupyter kernel LSST

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/06/27
- last update : 2024/06/27

## Documentation

- https://pipelines.lsst.io/middleware/index.html
- https://pipelines.lsst.io/middleware/faq.html#querycollections

example

           butler query-collections /sdf/group/rubin/repo/oga  

In [1]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2024_16/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2024_16/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2024_16/Spectractor 	setup


In [2]:
! eups list -s lsst_distrib

   gf193e5375e+09a23f6f84 	current w_2024_25 setup


In [3]:
import lsst.daf.butler as dafButler

In [4]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom
import pprint
# Let us also set `lsst.afw.display` to use the `matplotlib` backend:
#afwDisplay.setDefaultBackend('matplotlib')
afwDisplay.setDefaultBackend('firefly')

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm 
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from astropy.visualization import imshow_norm, MinMaxInterval, SqrtStretch
# Astropy
from astropy.visualization import ZScaleInterval, AsinhStretch

import seaborn as sns
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd


#xx-small
#x-small
#small
#medium
#large
#x-large
#xx-large

plt.rcParams["figure.figsize"] = (10,6)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

In [6]:
#repo =  "/sdf/group/rubin/repo/main"
repo =  'embargo_or4'
butler = dafButler.Butler(repo)
registry = butler.registry

In [7]:
#for c in sorted(registry.queryCollections()):
#    if "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553" in c:
#        print(c)

In [8]:
for c in sorted(registry.queryCollections()):
#    if "LSSTComCamSim" in c or "LATISS" in c:
    if "LSSTComCamSim" in c:
        print(c)

LSSTComCamSim/calib
LSSTComCamSim/calib/DM-42287
LSSTComCamSim/calib/DM-42287/unbounded
LSSTComCamSim/calib/DM-43441
LSSTComCamSim/calib/DM-43441/curated/19700101T000000Z
LSSTComCamSim/calib/DM-43441/unbounded
LSSTComCamSim/calib/DM-44910
LSSTComCamSim/calib/DM-44910/or4Flats/flat.20240620a
LSSTComCamSim/calib/unbounded
LSSTComCamSim/defaults
LSSTComCamSim/fgcmcal/lookuptable/DM-43441
LSSTComCamSim/nightlyValidation
LSSTComCamSim/nightlyValidation/0
LSSTComCamSim/nightlyValidation/1
LSSTComCamSim/prompt/output-2024-06-18
LSSTComCamSim/prompt/output-2024-06-18/ApPipe/prompt-proto-service-lsstcomcamsim-00039
LSSTComCamSim/prompt/output-2024-06-18/Preload/prompt-proto-service-lsstcomcamsim-00039
LSSTComCamSim/prompt/output-2024-06-21
LSSTComCamSim/prompt/output-2024-06-21/ApPipe/prompt-proto-service-lsstcomcamsim-00040
LSSTComCamSim/prompt/output-2024-06-21/Preload/prompt-proto-service-lsstcomcamsim-00040
LSSTComCamSim/prompt/output-2024-06-25
LSSTComCamSim/prompt/output-2024-06-25/ApPipe

In [9]:
my_collection = "LSSTComCamSim/nightlyValidation"

## Check postISRCCD

In [10]:
dataType = 'postISRCCD'

In [11]:
refs_ds=list(butler.registry.queryDatasets(datasetType=dataType,collections=my_collection))

In [12]:
for idx,ref in enumerate(refs_ds):
    print("------------------------------------------------------")
    print(ref)
    if idx>10:
        break

------------------------------------------------------
postISRCCD@{instrument: 'LSSTComCamSim', detector: 8, exposure: 7024061800042, band: 'i', day_obs: 20240618, group: '2024-06-19T00:35:19.639', physical_filter: 'i_06'} [sc=Exposure] (run=LSSTComCamSim/nightlyValidation/0 id=ad88a5b1-2697-4031-87c5-00067bb265da)
------------------------------------------------------
postISRCCD@{instrument: 'LSSTComCamSim', detector: 5, exposure: 7024061800042, band: 'i', day_obs: 20240618, group: '2024-06-19T00:35:19.639', physical_filter: 'i_06'} [sc=Exposure] (run=LSSTComCamSim/nightlyValidation/0 id=b7842bbf-8eff-472d-8d5c-a0ef24096665)
------------------------------------------------------
postISRCCD@{instrument: 'LSSTComCamSim', detector: 6, exposure: 7024061800042, band: 'i', day_obs: 20240618, group: '2024-06-19T00:35:19.639', physical_filter: 'i_06'} [sc=Exposure] (run=LSSTComCamSim/nightlyValidation/0 id=28dd5aa5-3374-485e-8d7b-6b980c473174)
-------------------------------------------------

In [13]:
ref = refs_ds[0]
postisrccd = butler.get(ref)
info=postisrccd.getInfo()
exp_id = info.getId()
the_title = f"postISRCCD {exp_id}"

In [14]:
# Plot the calexp we just retrieved
#plt.figure(figsize=(10,10))
afw_display = afwDisplay.Display(frame=1)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(postisrccd.image,title=the_title )
afw_display.setMaskTransparency(80)
    #plt.gca().axis('on')
#transform = AsinhStretch() + ZScaleInterval()
#scaledImage = transform(postisrccd.image.array)
    #scaledImage = np.flipud(scaledImage)
#plt.imshow(scaledImage,origin="lower",extent=extent,aspect='equal')
#    plt.title(the_title)
#plt.grid()
    #plt.gca().invert_xaxis()
#plt.show()


## Check Calexp

In [15]:
dataType = 'calexp'

In [16]:
refs_ds=list(butler.registry.queryDatasets(datasetType=dataType,collections=my_collection))

In [17]:
for idx,ref in enumerate(refs_ds):
    print("------------------------------------------------------")
    print(ref)
    if idx>10:
        break

------------------------------------------------------
calexp@{instrument: 'LSSTComCamSim', detector: 8, visit: 7024061800042, band: 'i', day_obs: 20240618, physical_filter: 'i_06'} [sc=ExposureF] (run=LSSTComCamSim/nightlyValidation/0 id=bd15b47a-fd1d-44ca-98d5-94c895812ae4)
------------------------------------------------------
calexp@{instrument: 'LSSTComCamSim', detector: 5, visit: 7024061800042, band: 'i', day_obs: 20240618, physical_filter: 'i_06'} [sc=ExposureF] (run=LSSTComCamSim/nightlyValidation/0 id=55fdc788-d273-42cd-a6d0-4a9412a54311)
------------------------------------------------------
calexp@{instrument: 'LSSTComCamSim', detector: 6, visit: 7024061800042, band: 'i', day_obs: 20240618, physical_filter: 'i_06'} [sc=ExposureF] (run=LSSTComCamSim/nightlyValidation/0 id=253fccc8-891f-44da-9177-0d76c38bd46d)
------------------------------------------------------
calexp@{instrument: 'LSSTComCamSim', detector: 7, visit: 7024061800042, band: 'i', day_obs: 20240618, physical_fil

In [18]:
ref = refs_ds[0]
calexp = butler.get(ref)
info=postisrccd.getInfo()
exp_id = info.getId()
the_title = f"calexp {exp_id}"

In [19]:
# Plot the calexp we just retrieved
#plt.figure(figsize=(10,10))
afw_display = afwDisplay.Display(frame=2)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp.image,title=the_title )
afw_display.setMaskTransparency(80)
    #plt.gca().axis('on')
#transform = AsinhStretch() + ZScaleInterval()
#scaledImage = transform(postisrccd.image.array)
    #scaledImage = np.flipud(scaledImage)
#plt.imshow(scaledImage,origin="lower",extent=extent,aspect='equal')
#    plt.title(the_title)
#plt.grid()
    #plt.gca().invert_xaxis()
#plt.show()


## preSources

dataType = 'preSource'

In [20]:
dataType = 'preSourceTable'

In [21]:
refs_ds=list(butler.registry.queryDatasets(datasetType=dataType,collections=my_collection))

In [22]:
for idx,ref in enumerate(refs_ds):
    print("------------------------------------------------------")
    print(ref)
    if idx>10:
        break

------------------------------------------------------
preSourceTable@{instrument: 'LSSTComCamSim', detector: 8, visit: 7024061800042, band: 'i', day_obs: 20240618, physical_filter: 'i_06'} [sc=DataFrame] (run=LSSTComCamSim/nightlyValidation/0 id=dbd52abf-1f44-484e-88f0-8d868efb37fe)
------------------------------------------------------
preSourceTable@{instrument: 'LSSTComCamSim', detector: 5, visit: 7024061800042, band: 'i', day_obs: 20240618, physical_filter: 'i_06'} [sc=DataFrame] (run=LSSTComCamSim/nightlyValidation/0 id=412f522e-1448-4ac8-a24d-8c07c32ff976)
------------------------------------------------------
preSourceTable@{instrument: 'LSSTComCamSim', detector: 6, visit: 7024061800042, band: 'i', day_obs: 20240618, physical_filter: 'i_06'} [sc=DataFrame] (run=LSSTComCamSim/nightlyValidation/0 id=3966cc91-23fb-49df-bac0-45428ac6651c)
------------------------------------------------------
preSourceTable@{instrument: 'LSSTComCamSim', detector: 7, visit: 7024061800042, band: 'i',

## preCcdVisitTable

In [23]:
dataType = 'preCcdVisitTable'

In [24]:
refs_ds=list(butler.registry.queryDatasets(datasetType=dataType,collections=my_collection))

In [25]:
for idx,ref in enumerate(refs_ds):
    print("------------------------------------------------------")
    print(ref)
    if idx>10:
        break

------------------------------------------------------
preCcdVisitTable@{instrument: 'LSSTComCamSim'} [sc=DataFrame] (run=LSSTComCamSim/nightlyValidation/0 id=a43d70ca-e7c9-4a66-acbf-7c107cb51a08)
------------------------------------------------------
preCcdVisitTable@{instrument: 'LSSTComCamSim'} [sc=DataFrame] (run=LSSTComCamSim/nightlyValidation/1 id=6a5d7234-4932-4791-b1d9-7c72e58af968)


## preVisitTable 

In [26]:
dataType = 'preVisitTable'

In [27]:
refs_ds=list(butler.registry.queryDatasets(datasetType=dataType,collections=my_collection))

In [28]:
for idx,ref in enumerate(refs_ds):
    print("------------------------------------------------------")
    print(ref)
    if idx>10:
        break

------------------------------------------------------
preVisitTable@{instrument: 'LSSTComCamSim'} [sc=DataFrame] (run=LSSTComCamSim/nightlyValidation/0 id=b77410a0-cb2b-4d87-a9ee-a4b5df926a79)
------------------------------------------------------
preVisitTable@{instrument: 'LSSTComCamSim'} [sc=DataFrame] (run=LSSTComCamSim/nightlyValidation/1 id=b86e755d-566d-4ee8-871e-3dc81918500a)


In [29]:
#butler =butlerUtils.makeDefaultLatissButler(extraCollections=[my_collection,])
# probably not working with OGA

In [30]:
dataId = {'day_obs': 20230912,'seq_num':264,'instrument':'LATISS','detector':0}

In [31]:
# with butler utils :
#butler.get('spectraction',dataId)

In [32]:
# without butler utils
spec= butler.get('spectractorSpectrum',dataId,collections=my_collection)

MissingDatasetTypeError: "Dataset type with name 'spectractorSpectrum' not found."

### Show all dataset types

In [ ]:
dst = list(butler.registry.queryDatasetTypes("*"))

In [ ]:
dst

In [ ]:
output_coll = "u/dagoret/spectro/emptyholo4003noflat/runs_20230912"

In [ ]:
for refs_ds in butler.registry.queryDatasets(datasetType='any',collections=output_coll):
    print(refs_ds)

In [ ]:
refs_ds

In [ ]:
refs_ds=list(butler.registry.queryDatasets(datasetType="*spe*",collections=output_coll))

In [ ]:
refs_ds

In [ ]:
refs_ds

In [ ]:
#/sdf/group/rubin/repo/main/u/dagoret/tests/test_20oct2022/20221020T143125Z/spectractorSpectrum/20220630/white/empty~holo4_003/2022063000060/82nlj40o9h1dcrhp.fits